# Kaleabe Negussie UGR/3411/12

## Dense layer (DenseLayer)

In [2]:
class DenseLayer:

    def __init__(self,n_features, n_neurons):
        self.weights = torch.rand((n_features,n_neurons))
        self.bias = torch.rand(n_neurons)

    def forward(self,inputs):
        self.output = torch.matmul(inputs,self.weights) + self.bias

## Loss(Loss_Catagorical)

In [3]:
class Loss_Catagorical:
    def __init__(self):
        self.output = float('inf')
    def forward(self,y_pred,y_true):
        '''
        takes two parameters y_pred:pytorch tensor the predicted outpurs
        y_true: tensor as one hot notation or numbers as groups indexed from one
        '''
        if y_pred.shape !=  y_true.shape:
            one_hot_notation = torch.zeros(y_pred.shape)
            one_hot_notation[range(len(y_pred)),y_true] = 1
        else:
            one_hot_notation = y_pred
        loss = -torch.sum(one_hot_notation * torch.log(y_pred)) / len(y_true)
        self.output = loss
        return loss

## Accuracy(Accuracy)

In [4]:
class Accuracy:
    def __init__(self):
        self.output = 0
    def forward(self,y_pred,y_true):
        if y_pred.shape != y_true.shape:
            one_hot_notation = torch.zeros(y_pred.shape)
            one_hot_notation[range(len(y_pred)),y_true] = 1
        else:
            one_hot_notation = y_true
        correct_values = y_pred==one_hot_notation
        correct_values = correct_values * one_hot_notation
        self.output = torch.sum(correct_values) / len(y_pred)
        return self.output

## Activation sigmoid (Activation_Sigmoid)

In [5]:
class Activation_Sigmoid:
    def forward(self,inputs):
        self.output = 1 / (1 + torch.exp(inputs*-1))
        return self.output

## Activation ReLU (Activation_ReLU)

In [6]:
class Activation_ReLU:
    def forward(self,inputs):
        self.output = torch.max(inputs,torch.tensor(0))
        return self.output

## Model

In [97]:
class ClassificationModel:
    def __init__(self,num_of_features:int,num_of_classes:int):
        self.layer1 = DenseLayer(num_of_features,4)
        self.activation1 = Activation_ReLU()
        self.output_layer = DenseLayer(4,num_of_classes)
        self.output_activation = Activation_Sigmoid()
        self.accuracy = Accuracy()
        self.errors = [float('inf')] * num_of_classes

    def forward_propagate(self,inputs):
        self.layer1.forward(inputs)
        self.activation1.forward(self.layer1.output)
        self.output_layer.forward(self.activation1.output)
        self.output_activation.forward(self.output_layer.output)
        self.inputs = inputs

    def loss_and_accuracy(self,output):
        self.true_value = output
        if self.output_activation.output.shape != output.shape:
            one_hot_notation = torch.zeros(self.output_activation.output.shape)
            one_hot_notation[range(len(self.output_activation.output)),output] = 1
            self.true_value = one_hot_notation

        loss = ((self.true_value - self.output_layer.output) ** 2) / 2
        loss = torch.mean(loss)
        accuracy = self.accuracy.forward(self.output_layer.output,self.true_value)
        return loss, accuracy


    def back_prop(self,lr):
        errors = -(self.true_value - self.output_layer.output) # d(y-output)/d(output)
        avg_errors = torch.sum(errors,keepdims=True,dim=0) / len(errors)
        avg_errors = torch.squeeze(avg_errors)
        avg_outputs = torch.sum(self.output_activation.output,keepdims=True,dim=0) / len(self.output_activation.output)
        avg_outputs = torch.squeeze(avg_outputs)
        avg_act1_outputs = torch.sum(self.activation1.output,keepdims=True,dim=0) / len(self.activation1.output)
        avg_act1_outputs = torch.squeeze(avg_act1_outputs)
        avg_inputs = torch.sum(self.inputs,keepdims=True,dim=0) / len(self.inputs)
        avg_inputs = torch.squeeze(avg_inputs)
        avg_layer1_output = torch.sum(self.layer1.output,keepdims=True,dim=0) / len(self.layer1.output)
        avg_layer1_output = torch.squeeze(avg_layer1_output)
        back = [None] * 4
        for j in range(4):
            back[j] = torch.tensor(lr) * avg_errors[j] * (avg_outputs[j] * (1-avg_outputs[j]))
        # second layer
        for i in range(4):
            self.output_layer.bias[i] -= back[i]
            for j in range(4):
                self.output_layer.weights[i][j] -= back[j] * avg_act1_outputs[i]


        #first layer
        for j in range(4):
            for k in range(4):
                self.layer1.bias[j] -= back[j] * self.output_layer.weights[j][k] * (1 if avg_layer1_output[j]>0 else 0)
                for i in range(2):
                    self.layer1.weights[i][j] -= back[k] * self.output_layer.weights[j][k] * avg_inputs[i] * (1 if avg_layer1_output[j]>0 else 0)


In [102]:
model = ClassificationModel(2,4)
x = torch.tensor([[1,2],[3,4],[4,5]],dtype=torch.float)
y = torch.tensor([1,1,0])
model.forward_propagate(x)
model.loss_and_accuracy(y)

loss = 0.1
error = float('inf')
iterations = 0
while loss < error:
    iterations += 1
    model.forward_propagate(x)
    model.back_prop(0.01)
    error,acc = model.loss_and_accuracy(y)

print("iterations:",iterations)
print("final output:",model.output_layer.output)
print("true vaues:",model.true_value)
print("accuracy:",acc)
print("final error:",error)


iterations: 389
final output: tensor([[0.6711, 0.7722, 0.0998, 0.4448],
        [0.6559, 0.7611, 0.0841, 0.4389],
        [0.6375, 0.7914, 0.1185, 0.4803]])
true vaues: tensor([[0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.]])
accuracy: tensor(0.)
final error: tensor(0.1000)
